In [205]:
from __future__ import division
import csv,sys
import numpy as np
import pandas as pd
import re
#import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time, locale
#Plot.ly
import plotly.plotly as py
from plotly.graph_objs import *
#from IPython import display as d
#import nvd3
from money import Money
### Librerias Propias
from load_data import obtener_provincia_cif
from load_data import obtener_tipo_cif
from load_data import obtener_tipo_cif_resumen
from load_data import string_to_timestamp
from load_data import listar_subvenciones_mes_grupo
from load_data import filtra_importes_grupo

#from nvd3 import discreteBarChart
%load_ext autoreload
%autoreload 2

#reload(sys)  # Reload does the trick!
#sys.setdefaultencoding('UTF8')

#nvd3.ipynb.initialize_javascript(use_remote=True)

#locale.setlocale(locale.LC_ALL, 'es_ES')
#locale.setlocale(locale.LC_ALL, 'C')
#locale.format("%10.2f", 1234.567, grouping=True) 
#pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones_2.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'
FILE_ENTES_LOCALES = 'data/entes_locales.csv'
FILE_ENTES_JJAA = 'data/entes_jjaa.csv'
FILE_FUNCIONAL_PRESUPUESTO = 'data/partidas_presupuesto_gasto.csv'
# Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fuentes de Información
----------------------

El siguiente análisis está basado en las siguientes fuentes de datos:
* Subvenciones. Portal de Datos Abiertos de la Junta de Andalucia
* Ayudas. ????
* Municipios. Instituto de Estadística y Cartografía de Andalucía
* Elecciones. Instituto de Estadística y Cartografía de Andalucía
* Entes Municipales. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. 
* Entes Autonómicos. Ministerio de Hacienda y Administraciones Públicas / Secretaría de Estado de Administraciones Públicas. (también disponible en Consejería de Hacienda y Administraciones Públicas de la Junta de Andalucía).
* Anuario de la Caixa 2014.
* Presupuesto 2015 de la Junta de Andalucía

Se pueden descargar de:
* Datos Abiertos / Secretaría de Estado de Administraciones Públicas. http://www.minhap.gob.es/es-ES/Datos%20Abiertos/Paginas/CatalogoRISPSecretariadeEstadodeAdministracionesPublicas.aspx
* http://serviciostelematicos.minhap.gob.es/PubInvCCAA/secciones/FrmListadoEntes.aspx?cdcdad=AX9BV45F&tiporg2=
* http://www.anuarieco.lacaixa.comunicacions.com/java/X?cgi=caixa.le_menuGeneral.pattern
Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf

In [206]:
#Read files
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')
entes_locales = pd.read_csv(FILE_ENTES_LOCALES, quotechar='"', sep=',')
entes_jjaa = pd.read_csv(FILE_ENTES_JJAA, quotechar='"', sep=',')
subvenciones = pd.read_csv(FILE_SUBVENCIONES, quotechar='"', sep='|', decimal=',')
funcional = pd.read_csv(FILE_FUNCIONAL_PRESUPUESTO, quotechar='"', sep='|', decimal=',').fillna('')

El importe total de las subvenciones includas en los datos abiertos es:

In [207]:
total_subvenciones = subvenciones['importe'].sum() 
m = Money(amount = total_subvenciones, currency='EUR')
print(m.format('es_ES'))

1.038.724.228,81 €


# Mejora de la información

(Data wranling / Data Munging)

A continuación se procede a la mejora de la información de base con otras fuentes de información (de las ya cargadas),
o de la análisis de la ya disponible.

In [208]:
subvenciones['mes_concesion_timestamp'] = subvenciones['fecha_concesion'].apply(lambda x: string_to_timestamp(x))
subvenciones['fecha_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.strptime(x, '%d/%m/%y'))
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: x.strftime('%Y/%m'))

In [209]:
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0])
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(obtener_tipo_cif)
subvenciones['tipo_beneficiario_res'] = subvenciones['id_beneficiario'].apply(obtener_tipo_cif_resumen)
subvenciones['tipo_beneficiario_cod'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])
subvenciones['provincia'] = subvenciones['id_beneficiario'].apply(obtener_provincia_cif)
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0])
subvenciones['provincia_cod_cif'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])

In [210]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion_timestamp').sum()
subvenciones_por_mes_grupo = subvenciones.groupby(['mes_concesion_timestamp','grupo_pres']).sum().reset_index()
subvenciones_por_finalidad = subvenciones.groupby('finalidad').sum().sort('importe',ascending=0).reset_index()
subvenciones_por_finalidad_beneficiario = subvenciones.groupby(['finalidad','tipo_beneficiario']).sum().sort('importe',ascending=0).reset_index()

In [211]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía


###¿Existen subvenciones no clasificadas?

El presupuesto marca el objetivo que se busca a muy grandes rasgos con las subvenciones.

In [217]:
programas_dict = subvenciones['programa'].to_dict()

# Existen registros de programas en otras columnas
programas_en_convocatoria = subvenciones['convocatoria'].str.contains('\s\d\d\D\s')
programas_en_convocatoria_res = subvenciones[programas_en_convocatoria]['convocatoria'].str.findall('\s\d\d\D\s').apply(lambda l: l[0])
for elemento in programas_en_convocatoria_res.index:
    programas_dict[elemento] = programas_en_convocatoria_res[elemento]




# Análisis Exploratorio

A continuación con la información ya revisada y ampliada se puede proceder a intentar responder a cuestiones sobre la información con que se cuenta.

## ¿Cómo se distribuye el gasto a través del tiempo?


In [137]:
lista_mes_concesion=list(subvenciones['mes_concesion'].drop_duplicates())
lista_mes_concesion.reverse()
lista_mes_concesion

data = Data([
    Bar(
        x=lista_mes_concesion,
        y=list(subvenciones_por_mes['importe']),
        text=lista_mes_concesion,
        marker=Marker(
            color='rgb(142, 124, 195)'
        )
    )
])

layout = Layout(
    title='Number of graphs made this week',
    font=Font(
        family='Raleway, sans-serif'
    ),
    showlegend=False,
    xaxis=XAxis(
        tickangle=-45
    ),
    yaxis=YAxis(
        zeroline=False,
        gridwidth=2
    ),
    bargap=0.05
)
fig = Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='bar-with-hover-text')
py.iplot(data, filename = 'evolucion-subveciones')

In [131]:
lista_meses_timestamp = subvenciones_por_mes_grupo['mes_concesion_timestamp'].unique()
lista_grupos = subvenciones_por_mes_grupo['grupo_pres'].unique()

In [132]:
def nueva_fila_mes_grupo(mes, grupo, importe = 0,id=0):
    new_data = { 'mes_concesion_timestamp' : [mes],
             'grupo_pres' : [grupo],
             'importe': [0], 
             'id': [id]}
    return pd.DataFrame(new_data)

In [133]:
subvenciones_mes_grupo_trabajo = subvenciones_por_mes_grupo.copy(deep=True)
contador_id = subvenciones_mes_grupo_trabajo['id'].argmax()

for mes in lista_meses_timestamp:
    for grupo in lista_grupos:
        query_mes_grupo = "mes_concesion_timestamp == %s & grupo_pres == '%s'" % (mes, grupo)
        resultados_query = subvenciones_mes_grupo_trabajo.query(query_mes_grupo)
#        print "%s -- %d " % (query_mes_grupo, len(resultados_query))
        if len(resultados_query) == 0:
            contador_id += 1
            subvenciones_mes_grupo_trabajo = pd.concat([subvenciones_mes_grupo_trabajo,nueva_fila_mes_grupo(mes,grupo,id=contador_id)])

#print(len(lista_meses_timestamp))
#print(subvenciones_mes_grupo_trabajo.groupby('grupo_pres').count())

Subvenciones por área del presupuesto
---------------------

Utilizando el identificador del presupuesto podemos agrupar las ayudas por el xxx del presupuesto de la Junta de Andalucía y su distribución a lo largo del tiempo.

El presupuesto de las comunidades autónomas según su función se clasifican en:
* Área,
* Politica,
* Función y
* Programa

Los datos de subvenciones tienen detallado hasta la función, no existe detalle del programa.

In [139]:
subvenciones_por_grupo = subvenciones_por_mes_grupo.groupby('grupo_pres').sum()
areas_presupuesto = funcional.query("POLITICA=='' and  PROGRAMA ==''").set_index('AREA')['DESCRIPCION'].to_dict()
#total_subvenciones
#print list(subvenciones_por_grupo['importe'])
trace1 = Bar(
    x= areas_presupuesto.values(),
    y=list(subvenciones_por_grupo['importe']),
    name='SF Zoo'
)
#trace2 = Bar(
#    x=['giraffes', 'orangutans', 'monkeys'],
#    y=[12, 18, 29],
#    name='LA Zoo'
#)
data = Data([trace1])


layout = Layout(
    title='Subvenciones por área del presupuesto',
    font=Font(
        family='Raleway, sans-serif'
    ),
    showlegend=False,
    xaxis=XAxis(
        tickangle=-45
    ),
    yaxis=YAxis(
        zeroline=False,
        gridwidth=2
    ),
    bargap=0.05
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')


['DEUDA PUBLICA',
 'SERVICIOS DE CARACTER GENERAL',
 'PROTECCION CIVIL Y SEGURIDAD CIUDADANA',
 'SEGURIDAD, PROTECCION Y PROMOCION SOCIAL',
 'PRODUCCI. DE BIENES PUBLICOS DE CARACTER SOCIAL',
 'PRODUCCION DE BIENES PUBLICOS DE CARACTER ECON.',
 'REGULACION ECONOMICA DE CARACTER GENERAL',
 'REG. ECONOMICA DE LA ACTIV. Y LOS SECT. PRODUCT.',
 'RELACIONES CON OTRAS ADMINISTRACIONES']

In [68]:
#subvenciones_por_finalidad_beneficiario['finalidad'].drop_duplicates()

In [13]:
chart4_name = "Subvenciones por area"
chart4 = nvd3.stackedAreaChart(name=chart4_name,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
chart4.set_containerheader("\n\n<h2>" + chart4_name + "</h2>\n\n")
xdata = list(subvenciones_por_mes_grupo.mes_concesion_timestamp.drop_duplicates())

for grupo in subvenciones_mes_grupo_trabajo.grupo_pres.order().drop_duplicates():
    chart4.add_serie(name=grupo, y=listar_subvenciones_mes_grupo(subvenciones_mes_grupo_trabajo,grupo), x=xdata)

chart4.margin_left = 100
chart4

Por Identificadores
-------------------

In [24]:
subvenciones['tipo_beneficiario'] = subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])
subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[0])#.drop_duplicates()

0        G
1        G
2        Q
3        G
4        G
5        G
6        G
7        G
8        G
9        G
10       Q
11       G
12       G
13       G
14       G
15       G
16       Q
17       G
18       7
19       P
20       P
21       P
22       G
23       P
24       P
25       P
26       P
27       P
28       P
29       P
        ..
71073    B
71074    3
71075    3
71076    B
71077    2
71078    3
71079    3
71080    3
71081    3
71082    2
71083    5
71084    2
71085    0
71086    2
71087    2
71088    3
71089    F
71090    2
71091    2
71092    B
71093    7
71094    5
71095    7
71096    2
71097    7
71098    7
71099    2
71100    7
71101    3
71102    P
Name: id_beneficiario, dtype: object

In [25]:
#subvenciones['id_beneficiario'].apply(lambda x: ("%s" % x)[1:3])#.drop_duplicates()
subvenciones['id_beneficiario'].apply(obtener_tipo_cif)

0                                             Asociaciones
1                                             Asociaciones
2        Organismos autónomos, estatales o no, y asimil...
3                                             Asociaciones
4                                             Asociaciones
5                                             Asociaciones
6                                             Asociaciones
7                                             Asociaciones
8                                             Asociaciones
9                                             Asociaciones
10       Organismos autónomos, estatales o no, y asimil...
11                                            Asociaciones
12                                            Asociaciones
13                                            Asociaciones
14                                            Asociaciones
15                                            Asociaciones
16       Organismos autónomos, estatales o no, y asimil.

In [26]:
subvenciones.query("tipo_beneficiario_cod =='P' and provincia == 'Madrid'")['id_beneficiario']

Series([], Name: id_beneficiario, dtype: object)

In [27]:
subvenciones.groupby(['provincia']).count()
#subvenciones.query("provincia =='Teruel'")['id_beneficiario']

,id,base_reguladora,convocatoria,consejeria,organo_gestor,tipo_subvencion,programa,denominacion_programa,importe,finalidad,id_beneficiario,nombre_beneficiario,fecha_concesion,partida_pres,mes_concesion,grupo_pres,tipo_beneficiario,tipo_beneficiario_cod,provincia_cod_cif
provincia,,,,,,,,,,,,,,,,,,,
NA,71103,71103,71103,71103,71103,71103,64343,64343,71103,71103,71094,71103,71103,566,71103,71103,71103,71103,71103


In [28]:
subvencione

0   2015-03-27
1   2015-03-27
2   2015-03-27
3   2015-03-27
4   2015-03-27
Name: fecha_concesion, dtype: datetime64[ns]

In [151]:
subvenciones.groupby('grupo_pres').sum()['importe']

grupo_pres
1    5.003047e+06
2    1.229375e+05
3    3.965595e+08
4    1.021258e+08
5    7.480284e+07
6    3.500000e+05
7    2.883880e+08
8    8.642083e+07
n    8.495133e+07
Name: importe, dtype: float64

{0: '31G',
 1: '31G',
 2: '31G',
 3: '31G',
 4: '31G',
 5: '31G',
 6: '31G',
 7: '31G',
 8: '31G',
 9: '31G',
 10: '31G',
 11: '31G',
 12: '31G',
 13: '31G',
 14: '31G',
 15: '31G',
 16: '31G',
 17: '31G',
 18: '32F',
 19: '43A',
 20: '43A',
 21: '43A',
 22: '71B',
 23: '31B',
 24: '31B',
 25: '31E',
 26: '31B',
 27: '31G',
 28: '31G',
 29: '31G',
 30: '31G',
 31: '31G',
 32: '31G',
 33: '31G',
 34: '31G',
 35: '31G',
 36: '31G',
 37: '31G',
 38: '31G',
 39: '32F',
 40: '32F',
 41: '32F',
 42: '32F',
 43: '31R',
 44: '31G',
 45: '31G',
 46: '31G',
 47: '31G',
 48: '31G',
 49: '31G',
 50: '31G',
 51: '31G',
 52: '31G',
 53: '31B',
 54: '31B',
 55: '31B',
 56: '31B',
 57: '31B',
 58: '22B',
 59: '41D',
 60: '41D',
 61: '41D',
 62: '41D',
 63: '41D',
 64: '41D',
 65: '41D',
 66: '41D',
 67: '41D',
 68: '41D',
 69: '41D',
 70: '41D',
 71: '41D',
 72: '41D',
 73: '41D',
 74: '41D',
 75: '41D',
 76: '41D',
 77: '41D',
 78: '41D',
 79: '41D',
 80: '41D',
 81: '41D',
 82: '41D',
 83: '41D',
 8